# Sentiment Analysis on Product Reviews using LSTM 
<hr>

### Steps
<ol type="1">
    <li>Load the dataset </li>
    <li>Clean Dataset</li>
    <li>Encode Sentiments</li>
    <li>Split Dataset</li>
    <li>Tokenize and Pad/Truncate Reviews</li>
    <li>Build Architecture/Model</li>
    <li>Train and Test</li>
</ol>

<hr>
<i>Importing all the libraries needed</i>

In [1]:
!pip install tensorflow

In [2]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split # for splitting dataset
  # load saved model
import re

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model 

<hr>
<i>Preview dataset</i>

In [4]:
dataa = pd.read_csv('product.csv')

print(dataa)

                                                   review sentiment
0       great cooler excellent air flow and for this p...  positive
1                   best budget 2 fit cooler nice cooling  positive
2       the quality is good but the power of air is de...  positive
3                       very bad product its a only a fan  negative
4                                           ok ok product   neutral
...                                                   ...       ...
205047                                       good product  positive
205048                                               nice  positive
205049                        very nice and fast delivery  positive
205050                                    awesome product  positive
205051  very good but mixing bowl not included is one ...   neutral

[205052 rows x 2 columns]


<hr>
<b>Stop Word</b> is a commonly used words in a sentence, usually a search engine is programmed to ignore this words (i.e. "the", "a", "an", "of", etc.)

<i>Declaring the english stop words</i>

In [4]:
english_stops = set(stopwords.words('english'))

<hr>

### Load and Clean Dataset

In the original dataset, the reviews are still dirty. There are still html tags, numbers, uppercase, and punctuations. This will not be good for training, so in <b>load_dataset()</b> function, beside loading the dataset using <b>pandas</b>, I also pre-process the reviews by removing html tags, non alphabet (punctuations and numbers), stop words, and lower case all of the reviews.

### Encode Sentiments
In the same function, I also encode the sentiments into integers (0 and 1). Where 0 is for negative sentiments and 1 is for positive sentiments.

In [44]:
def load_dataset():
    df = pd.read_csv('product..csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case
    
    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)
    y_data = y_data.replace('neutral',0.5)

    return x_data, y_data

x_data, y_data = load_dataset()


<hr>

### Split Dataset
In this work, I decided to split the data into 80% of Training and 20% of Testing set using <b>train_test_split</b> method from Scikit-Learn. By using this method, it automatically shuffles the dataset. We need to shuffle the data because in the original dataset, the reviews and sentiments are in order, where they list positive reviews first and then negative reviews. By shuffling the data, it will be distributed equally in the model, so it will be more accurate for predictions.

In [45]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
#print(x_train, '\n')
#print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
Test Set
41331    1
3448     1
44162    1
1985     0
37208    0
        ..
33775    1
1165     1
43062    0
38693    1
21316    1
Name: sentiment, Length: 40000, dtype: int64 

14355    0
16019    0
40781    1
6209     1
10020    0
        ..
24111    0
39220    1
49434    0
18616    0
1763     0
Name: sentiment, Length: 10000, dtype: int64


<hr>
<i>Function for getting the maximum review length, by calculating the mean of all the reviews length (using <b>numpy.mean</b>)</i>

In [46]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

<hr>

### Tokenize and Pad/Truncate Reviews
A Neural Network only accepts numeric data, so we need to encode the reviews. I use <b>tensorflow.keras.preprocessing.text.Tokenizer</b> to encode the reviews into integers, where each unique word is automatically indexed (using <b>fit_on_texts</b> method) based on <b>x_train</b>. <br>
<b>x_train</b> and <b>x_test</b> is converted into integers using <b>texts_to_sequences</b> method.

Each reviews has a different length, so we need to add padding (by adding 0) or truncating the words to the same length (in this case, it is the mean of all reviews length) using <b>tensorflow.keras.preprocessing.sequence.pad_sequences</b>.


<b>post</b>, pad or truncate the words in the back of a sentence<br>
<b>pre</b>, pad or truncate the words in front of a sentence

In [47]:
import pickle

In [48]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
pickle.dump(token,open('token.pkl','wb'))
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[  390   203    28 ...     0     0     0]
 [  405   162  1986 ...   246   376    21]
 [    1  1589   408 ...     0     0     0]
 ...
 [    1   325  2635 ...     0     0     0]
 [32096 12153   425 ...     0     0     0]
 [    1 92648  2144 ...     0     0     0]] 

Encoded X Test
 [[  894   436  1587 ...     0     0     0]
 [    2  1060     1 ...     0     0     0]
 [  736   697  1088 ...     0     0     0]
 ...
 [ 3158   755   821 ...     0     0     0]
 [  205    70    87 ... 15424     7   489]
 [    1    12  5731 ...     0     0     0]] 

Maximum review length:  130


<hr>

### Build Architecture/Model
<b>Embedding Layer</b>: in simple terms, it creates word vectors of each word in the <i>word_index</i> and group words that are related or have similar meaning by analyzing other words around them.

<b>LSTM Layer</b>: to make a decision to keep or throw away data by considering the current input, previous output, and previous memory. There are some important components in LSTM.
<ul>
    <li><b>Forget Gate</b>, decides information is to be kept or thrown away</li>
    <li><b>Input Gate</b>, updates cell state by passing previous output and current input into sigmoid activation function</li>
    <li><b>Cell State</b>, calculate new cell state, it is multiplied by forget vector (drop value if multiplied by a near 0), add it with the output from input gate to update the cell state value.</li>
    <li><b>Ouput Gate</b>, decides the next hidden state and used for predictions</li>
</ul>

<b>Dense Layer</b>: compute the input with the weight matrix and bias (optional), and using an activation function. I use <b>Sigmoid</b> activation function for this work because the output is only 0 or 1.

The optimizer is <b>Adam</b> and the loss function is <b>Binary Crossentropy</b> because again the output is only 0 and 1, which is a binary number.

In [49]:
# ARCHITECTURE
EMBED_DIM = 32
LSTM_OUT = 64

model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 130, 32)           2964768   
                                                                 
 lstm_1 (LSTM)               (None, 64)                24832     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2989665 (11.40 MB)
Trainable params: 2989665 (11.40 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


<hr>

### Training


In [50]:
checkpoint = ModelCheckpoint(
    'models/LSTM.h5',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)

In [51]:
model.fit(x_train, y_train, batch_size = 128, epochs = 5, callbacks=[checkpoint])

Epoch 1/5
313/313 [==============================] - ETA: 0s - loss: 0.4734 - accuracy: 0.7532
Epoch 1: accuracy improved from -inf to 0.75318, saving model to models\LSTM.h5
313/313 [==============================] - 47s 138ms/step - loss: 0.4734 - accuracy: 0.7532


c:\Users\krishnendu\miniconda3\envs\env\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/5
313/313 [==============================] - ETA: 0s - loss: 0.2124 - accuracy: 0.9227
Epoch 2: accuracy improved from 0.75318 to 0.92275, saving model to models\LSTM.h5
313/313 [==============================] - 41s 130ms/step - loss: 0.2124 - accuracy: 0.9227
Epoch 3/5
313/313 [==============================] - ETA: 0s - loss: 0.1255 - accuracy: 0.9613
Epoch 3: accuracy improved from 0.92275 to 0.96130, saving model to models\LSTM.h5
313/313 [==============================] - 42s 133ms/step - loss: 0.1255 - accuracy: 0.9613
Epoch 4/5
313/313 [==============================] - ETA: 0s - loss: 0.0789 - accuracy: 0.9764
Epoch 4: accuracy improved from 0.96130 to 0.97640, saving model to models\LSTM.h5
313/313 [==============================] - 41s 132ms/step - loss: 0.0789 - accuracy: 0.9764
Epoch 5/5
313/313 [==============================] - ETA: 0s - loss: 0.0508 - accuracy: 0.9869
Epoch 5: accuracy improved from 0.97640 to 0.98693, saving model to models\LSTM.h5
313/313 [===

In [52]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(x_train, y_train)

# Print accuracy
print(f"Final Model Accuracy: {accuracy * 100:.2f}%")


1250/1250 [==============================] - 34s 27ms/step - loss: 0.0327 - accuracy: 0.9929
Final Model Accuracy: 99.29%


---

### Load Saved Model

Load saved model and use it to predict a movie review statement's sentiment (positive or negative).

In [53]:
loaded_model = load_model('models/LSTM.h5')

Receives a review as an input to be predicted

In [65]:
review = str(input('Product Review: '))

The input must be pre processed before it is passed to the model to be predicted

In [66]:
# Pre-process input
regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in english_stops]
filtered = ' '.join(filtered)
filtered = [filtered.lower()]

print('Filtered: ', filtered)

Cleaned:  This product is very good
Filtered:  ['this product good']


Once again, we need to tokenize and encode the words. I use the tokenizer which was previously declared because we want to encode the words based on words that are known by the model.

In [67]:
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)

[[   9 2089    8    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]]


This is the result of the prediction which shows the **confidence score** of the review statement.

In [68]:
result = loaded_model.predict(tokenize_words)
print(result)

1/1 [==============================] - 0s 429ms/step
[[0.14360192]]


In [63]:
if result >= 0.8:
    print('*****')
elif result>0.65:
    print('****')
elif result>0.5:
    print('***')
elif result>0.3:
    print('**')
else:
    print('*')

*****
